In [1]:
import os
import sys
import argparse
import pumpp
import shutil
import pickle
import librosa as lr
import glob

from joblib import Parallel, delayed
from pumpp.feature import Mel

sys.path.append('/home/ci411/pcen-t-varying/')
from pcen_t.utils import *
from pcen_t.pcen_pump import *

URBANSED_CLASSES = ['air_conditioner',
                    'car_horn',
                    'children_playing',
                    'dog_bark',
                    'drilling',
                    'engine_idling',
                    'gun_shot',
                    'jackhammer',
                    'siren',
                    'street_music']

In [2]:
%load_ext autoreload

In [3]:
%autoreload

In [2]:
def build_pump(sr, hop_length, n_fft, n_mels, n_t_constants=10, save_pump=None):
    pcen_t = PCEN_T(name='PCEN', sr=sr, n_fft=n_fft, hop_length=hop_length,\
                    n_t_constants=n_t_constants, n_mels=n_mels)
    
    p_tag = pumpp.task.StaticLabelTransformer(name='static',
                                              namespace='tag_open',
                                              labels=URBANSED_CLASSES)

    p_dtag = pumpp.task.DynamicLabelTransformer(name='dynamic',
                                                namespace='tag_open',
                                                labels=URBANSED_CLASSES,
                                                sr=sr,
                                                hop_length=hop_length)
    pump = pumpp.Pump(pcen_t, p_tag, p_dtag)
    
    if save_pump is not None:
        with open(os.path.join(save_pump, 'pump.pkl'), 'wb') as fd:
            pickle.dump(pump, fd)

    return pump

In [3]:
def build_pump_mel(sr, hop_length, n_fft, n_mels, save_pump=None):
    mel = Mel(name='Mel', sr=sr, hop_length=hop_length,\
                 n_fft=n_fft, n_mels=n_mels, log=True)
    
    p_tag = pumpp.task.StaticLabelTransformer(name='static',
                                              namespace='tag_open',
                                              labels=URBANSED_CLASSES)

    p_dtag = pumpp.task.DynamicLabelTransformer(name='dynamic',
                                                namespace='tag_open',
                                                labels=URBANSED_CLASSES,
                                                sr=sr,
                                                hop_length=hop_length)
    pump = pumpp.Pump(mel, p_tag, p_dtag)
    
    if save_pump is not None:
        with open(os.path.join(save_pump, 'pump.pkl'), 'wb') as fd:
            pickle.dump(pump, fd)

    return pump

In [6]:
sr = 22050 #44100 in autopool
hop_length = 1024
n_fft = 1024 #2048 in autopool
n_mels = 128

save_pump_pcen = '/scratch/ci411/pcen/pumps/pcen/'
save_pump_mel = '/scratch/ci411/pcen/pumps/mel/'

In [9]:
pcen_pump = build_pump(sr, hop_length, n_fft, n_mels, n_t_constants=10, save_pump=save_pump_pcen)
mel_pump = build_pump_mel(sr, hop_length, n_fft, n_mels,  save_pump=save_pump_mel)


In [12]:
audio_path = '/scratch/ci411/pcen/reverb_pitch_URBAN-SED/audio'
jams_path = '/scratch/ci411/pcen/reverb_pitch_URBAN-SED/annotations'

audio_files = glob.glob(os.path.join(audio_path, 'validate/*.wav'))
audio_files

[]

In [ ]:
for _ in range(100):
    file = np.random.choice(audio_files)
    test_file = file.split('/')[-1].split('.')[0]
    test_audio = os.path.join(audio_path, 'validate', test_file + '.wav')
    test_jams = os.path.join(jams_path, 'validate', test_file + '.jams')
    y, sr = lr.load(test_audio)

    mel_data = mel_pump.transform(test_audio, test_jams)
    pcen_data = pcen_pump.transform(test_audio, test_jams)
    print(mel_data['Mel/mag'].shape[1], pcen_data['PCEN/mag'].shape[1], mel_data['Mel/mag'].shape[1]==pcen_data['PCEN/mag'].shape[1], mel_data['Mel/mag'].shape[1]-pcen_data['PCEN/mag'].shape[1])
    #assert mel_data['Mel/mag'].shape[1]==pcen_data['PCEN/mag'].shape[1]

327 329 False -2
322 324 False -2
322 324 False -2
327 329 False -2
229 231 False -2
327 329 False -2
322 324 False -2
322 324 False -2
229 231 False -2
215 217 False -2
322 324 False -2
327 329 False -2
322 324 False -2
322 324 False -2
322 324 False -2
327 329 False -2
223 225 False -2
229 231 False -2
223 225 False -2
322 324 False -2
229 231 False -2
322 324 False -2
223 225 False -2
322 324 False -2
229 231 False -2
322 324 False -2
223 225 False -2
322 324 False -2
215 217 False -2
327 329 False -2
322 324 False -2
223 225 False -2
223 225 False -2
322 324 False -2
327 329 False -2
322 324 False -2
215 217 False -2
322 324 False -2
229 231 False -2
322 324 False -2
322 324 False -2
223 225 False -2
322 324 False -2
322 324 False -2
223 225 False -2
229 231 False -2
322 324 False -2
215 217 False -2
229 231 False -2
327 329 False -2
223 225 False -2
322 324 False -2
327 329 False -2
229 231 False -2
322 324 False -2


In [ ]:
test_file = 'soundscape_validate_bimodal662'
semitone = '0'
test_audio = os.path.join(audio_path, 'validate', test_file + '_{}.wav'.format(semitone))
test_jams = os.path.join(jams_path, 'validate', test_file + '_{}.jams'.format(semitone))
y, sr = lr.load(test_audio)

mel_data = mel_pump.transform(test_audio, test_jams)

pcen_data = pcen_pump.transform(test_audio, test_jams)

assert mel_data['Mel/mag'].shape[1]==pcen_data['PCEN/mag'].shape[1]

In [ ]:
from librosa.display import specshow
specshow(pcen_data['PCEN/mag'][0,:,:,2].T)